# Pasos

1. Cargar los datos
2. probabilidades iniciales, eta inicial?
3. Definir funciones
4. Actualizar valores




In [1]:
from pathlib import Path

from astropy.table import Table

from sgrmemb.loader import get_arrays_vasiliev
from sgrmemb.settings import FileConfig
from sgrmemb.plots import make_prob_slices_plots_modular

from sgrmemb.densitygen import (
    gen_ast,
    gen_ast_memb2,
    gen_pos_memb,
    gen_pos_field,
    gen_phot,
)
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt


In [2]:
vasiliev_data = get_arrays_vasiliev(FileConfig.VASILIEV_DATA)

In [ ]:
from proc.pipeline import load, get_subspace_data

SPACE_PARAMS = {
    "pos_αδ": ["ra", "dec"],
    "pos_lb": ["l", "b"],
    "pos_xy": ["SgrX", "SgrY"],
    "pm": ["pmra", "pmdec"],
    "pm_corr": ["pmra_corr", "pmdec_corr"],
    "cmd": ["mag_J-mag_Ks", "mag_Ks"],
    "ccd": ["mag_J-mag_Ks", "mag_H-mag_Ks"],
    "cmd_gaia": ["bp_rp", "phot_g_mean_mag"],
}

filepath = Path("/home/jorge/Documents/data/sgr/base_sample/lowbulge/60_merged/vvv_x_vvvx_pm+parallax_merged.fits")

df = load(filepath)
train_data = get_subspace_data(df, SPACE_PARAMS)

Primer paso, calcular probabilidades desde los PM y/o paralaje

In [ ]:
eta = 0.001

prob_xi_memb = df["prob_xi_memb"]
prob_xi_field = df["prob_xi_field"]

total_likelihood = eta * prob_xi_memb + (1 - eta) * prob_xi_field
q_memb_i = eta * prob_xi_memb / total_likelihood
q_field_i = (1 - eta) * prob_xi_field / total_likelihood
mask = np.isnan(q_memb_i)
eta = np.average(q_memb_i[~mask])
print(eta)

In [ ]:
for iteration in range(101):
    total_likelihood = eta * prob_xi_memb + (1 - eta) * prob_xi_field
    q_memb_i = eta * prob_xi_memb / total_likelihood
    q_field_i = (1 - eta) * prob_xi_field / total_likelihood
    mask = np.isnan(q_memb_i)
    eta = np.average(q_memb_i[~mask])
    if iteration % 10 == 0:
        print(iteration, eta)

Graficar

In [ ]:
# Results
plt.hist(q_memb_i, bins=50)
plt.close()
make_prob_slices_plots_modular(
    labeled_data=(
        ("pos", train_data["pos_lb"]),
        ("ast", train_data["pm"]),
        ("cmd", train_data["cmd"]),
    ),
    prob=q_memb_i,
    n_slices=4,
    save_path=None,
)

Actualizar modelos

In [ ]:
mask_nan_memb = np.isnan(q_memb_i)
mask_nan_field = np.isnan(q_field_i)

pos_memb = gen_pos_memb(X_train=vasiliev_data["pos_xy"])
pos_field = gen_pos_field(
    X_train=train_data["pos_lb"], weights=q_field_i, n_bins=10, n_intervals=2
)
phot_memb = gen_phot(X_train=train_data["cmd"][~mask_nan_memb], weights=q_memb_i[~mask_nan_memb])
phot_field = gen_phot(X_train=train_data["cmd"][~mask_nan_field], weights=q_field_i[~mask_nan_field])

In [ ]:
# Expectation
prob_xi_memb = df["prob_xi_memb"] * pos_memb(train_data["pos_xy"]) * phot_memb(train_data["cmd"])
prob_xi_field = df["prob_xi_field"] * pos_field(train_data["pos_lb"]) * phot_field(train_data["cmd"])

In [ ]:
total_likelihood = eta * prob_xi_memb + (1 - eta) * prob_xi_field
q_memb_i = eta * prob_xi_memb / total_likelihood
q_field_i = (1 - eta) * prob_xi_field / total_likelihood
mask = np.isnan(q_memb_i)
eta = np.average(q_memb_i[~mask])
print(eta)

Graficar

In [ ]:
make_prob_slices_plots_modular(
    labeled_data=(
        ("pos", train_data["pos_lb"]),
        ("ast", train_data["pm"]),
        ("cmd", train_data["cmd"]),
    ),
    prob=q_memb_i,
    n_slices=4,
    save_path=FileConfig.OUTPUT_PATH / "histograms.png",
)